<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/01_preprocessing/FF%2B%2B_download_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install insightface onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.0 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071489 sha256=0df0a04305c61a12ddc0221fbdd6168700c430b50911045b51655894415f08b8
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [2]:
import cv2
import numpy as np
from google.colab import drive
import os
import json
import urllib.request
import tempfile
import random
from tqdm import tqdm
from insightface.app import FaceAnalysis

import shutil
import zipfile

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# --- 설정 수정 ---
LOCAL_BASE_PATH = "/content/FF_frames_local"  # 코랩 로컬 경로에서 작업
GDRIVE_SAVE_BASE = "/content/drive/MyDrive/HECTO/Dataset/final_files" # 결과물이 저장될 구드 폴더
GDRIVE_ZIP_PATH = os.path.join(GDRIVE_SAVE_BASE, "FF_frames.zip")
JSON_DB_PATH = os.path.join(GDRIVE_SAVE_BASE, "ff_face_metadata.json")
VERIFY_SAVE_PATH = os.path.join(GDRIVE_SAVE_BASE, "FF_Verification")

# 구드에 저
FINAL_SAVE_PATH = "/content/drive/MyDrive/HECTO/Dataset/FF_Frames_Final"

NUM_FRAMES = 15
SERVER_URL = "http://kaldir.vc.in.tum.de/faceforensics/v3/"
FILELIST_URL = SERVER_URL + "misc/filelist.json"
MANIPULATIONS = ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']
COMPRESSION = 'c23'

In [5]:
# 아이디 대역별로 조작방식 설정하도록 수정
def get_id_matched_dataset():
    # 1. Real: 000.mp4 ~ 999.mp4까지 1,000개 강제 생성
    real_list = [f"{i:03d}.mp4" for i in range(1000)]

    # 2. Fake: 서버의 500개 쌍을 정방향/역방향으로 풀어 1,000개 생성
    print("서버에서 Fake 매칭 리스트를 가져오는 중...")
    with urllib.request.urlopen(FILELIST_URL) as url:
        file_pairs = json.loads(url.read().decode())

    fake_dict = {m: [] for m in MANIPULATIONS}

    for pair in file_pairs:
        # pair는 ['000', '003'] 같은 형태임
        id_a, id_b = pair[0], pair[1]

        # 방향 1: id_a를 배경(Target)으로 사용 (id_a_id_b.mp4)
        try:
            int_a = int(id_a)
            if int_a < 1000:
                method = MANIPULATIONS[min(int_a // 250, 3)]
                fake_dict[method].append(f"{id_a}_{id_b}.mp4")
        except: pass

        # 방향 2: id_b를 배경(Target)으로 사용 (id_b_id_a.mp4)
        try:
            int_b = int(id_b)
            if int_b < 1000:
                method = MANIPULATIONS[min(int_b // 250, 3)]
                fake_dict[method].append(f"{id_b}_{id_a}.mp4")
        except: pass

    # 각 카테고리별로 정렬하여 순서 보장
    for m in MANIPULATIONS:
        fake_dict[m].sort()

    print(f"✅ 최종 리스트 확정: Real {len(real_list)}개 / Fake 총 {sum(len(v) for v in fake_dict.values())}개")
    return real_list, fake_dict

In [6]:
# RetinaFace 초기화 (랜드마크 추출을 위해 landmark2d 추가)
detector = FaceAnalysis(allowed_modules=['detection', 'landmark_2d'], providers=['CUDAExecutionProvider'])
detector.prepare(ctx_id=0, det_size=(640, 640))

# # providers를 ['CPUExecutionProvider']로 변경하고 ctx_id=-1로 설정
# detector = FaceAnalysis(allowed_modules=['detection', 'landmark_2d'], providers=['CPUExecutionProvider'])
# detector.prepare(ctx_id=-1, det_size=(640, 640))

def get_aligned_face(image, face_info, target_size=(256, 256)):
    try:
        h_img, w_img = image.shape[:2]
        landmarks = getattr(face_info, 'kps', None)
        bbox = getattr(face_info, 'bbox', None)
        if bbox is None: return None

        x1, y1, x2, y2 = bbox.astype(int)
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

        # 1. 정렬 시도 (눈 위치가 있는 경우)
        if landmarks is not None and len(landmarks) >= 2:
            left_eye, right_eye = landmarks[0], landmarks[1]
            dy, dx = right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]
            angle = np.degrees(np.arctan2(dy, dx))

            M = cv2.getRotationMatrix2D((float(center_x), float(center_y)), angle, 1.0)
            image = cv2.warpAffine(image, M, (w_img, h_img), flags=cv2.INTER_CUBIC)

        # 2. 정사각 크롭 영역 계산 (애초에 정사각으로 잡기)
        w_box, h_box = x2 - x1, y2 - y1
        # 얼굴의 더 긴 쪽을 기준으로 마진(30%)을 포함한 정사각 변 길이 결정
        side_length = int(max(w_box, h_box) * 1.3)

        half_side = side_length // 2
        nx1, ny1 = center_x - half_side, center_y - half_side
        nx2, ny2 = center_x + half_side, center_y + half_side

        # 이미지 경계를 벗어나는 경우 패딩 처리 (검은색으로 채움)
        # 이렇게 해야 '짜부'되지 않고 정사각 형태를 유지합니다.
        pad_x1 = max(0, -nx1)
        pad_y1 = max(0, -ny1)
        pad_x2 = max(0, nx2 - w_img)
        pad_y2 = max(0, ny2 - h_img)

        nx1, ny1 = max(0, nx1), max(0, ny1)
        nx2, ny2 = min(w_img, nx2), min(h_img, ny2)

        face_crop = image[ny1:ny2, nx1:nx2]

        # 부족한 부분을 검은색으로 채워 정사각 유지
        if pad_x1 > 0 or pad_y1 > 0 or pad_x2 > 0 or pad_y2 > 0:
            face_crop = cv2.copyMakeBorder(face_crop, pad_y1, pad_y2, pad_x1, pad_x2,
                                           cv2.BORDER_CONSTANT, value=[0, 0, 0])

        return cv2.resize(face_crop, target_size)
    except:
        return None

def resize_with_letterbox(image, target_size=(256, 256)):
    if image.size == 0: return np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)
    h, w = image.shape[:2]
    scale = min(target_size[0]/w, target_size[1]/h)
    nw, nh = int(w * scale), int(h * scale)

    img_resized = cv2.resize(image, (nw, nh))
    canvas = np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)
    dx, dy = (target_size[0]-nw)//2, (target_size[1]-nh)//2
    canvas[dy:dy+nh, dx:dx+nw] = img_resized
    return canvas

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:02<00:00, 102996.42KB/s]


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvid

In [7]:
def process_and_extract_combined(video_names, sub_path, category_name, label, detector, is_real=True):
    # 구글 드라이브 최종 저장 경로 설정
    save_dir = os.path.join(FINAL_SAVE_PATH, category_name)
    os.makedirs(save_dir, exist_ok=True)

    # JSON DB 로드 (기존 데이터가 있으면 불러와서 이어서 작업)
    metadata = {}
    if os.path.exists(JSON_DB_PATH):
        with open(JSON_DB_PATH, 'r') as f:
            metadata = json.load(f)

    for v_name in tqdm(video_names, desc=f"Processing {category_name}"):
        video_id = v_name.split('.')[0]
        video_folder = os.path.join(save_dir, video_id)

        # 🔥 [이어하기 로직] 이미 폴더가 있고 프레임이 다 차있으면 스킵
        if os.path.exists(video_folder) and len(os.listdir(video_folder)) >= NUM_FRAMES:
            continue

        os.makedirs(video_folder, exist_ok=True)
        video_url = SERVER_URL + sub_path + v_name
        target_id = video_id.split('_')[0] if not is_real else video_id

        with tempfile.NamedTemporaryFile(suffix='.mp4') as temp_v:
            try:
                urllib.request.urlretrieve(video_url, temp_v.name)
                cap = cv2.VideoCapture(temp_v.name)
                total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if total <= 0: continue
                interval = max(1, total // NUM_FRAMES)

                last_face_info = None
                video_faces_found = False
                frames_cache = []
                face_crops_temp = []
                current_video_metadata = [None] * NUM_FRAMES

                for i in range(NUM_FRAMES):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
                    ret, frame = cap.read()
                    if not ret or frame is None:
                        face_crops_temp.append(None)
                        continue

                    frames_cache.append(frame)
                    face_to_save = None
                    used_info = None

                    if is_real:
                        # [REAL] GPU 가속 얼굴 검출
                        faces = detector.get(frame)
                        if faces:
                            used_info = sorted(faces, key=lambda x: (getattr(x, 'score', 0.0) or 0.0), reverse=True)[0]
                            face_to_save = get_aligned_face(frame, used_info)

                            if face_to_save is not None:
                                # 소급 적용
                                if not video_faces_found and i > 0:
                                    for prev_idx in range(len(face_crops_temp)):
                                        if face_crops_temp[prev_idx] is None:
                                            retro_crop = get_aligned_face(frames_cache[prev_idx], used_info)
                                            face_crops_temp[prev_idx] = retro_crop
                                            current_video_metadata[prev_idx] = {
                                                "bbox": used_info.bbox.tolist(),
                                                "kps": used_info.kps.tolist() if hasattr(used_info, 'kps') else None
                                            }
                                last_face_info = used_info
                                video_faces_found = True

                        if face_to_save is None and last_face_info is not None:
                            face_to_save = get_aligned_face(frame, last_face_info)
                            used_info = last_face_info
                    else:
                        # [FAKE] JSON 좌표 재활용 (구드에서 불러온 metadata 사용)
                        if target_id in metadata and i < len(metadata[target_id]):
                            info = metadata[target_id][i]
                            if info:
                                class FaceInfo: pass
                                used_info = FaceInfo()
                                used_info.bbox = np.array(info['bbox'])
                                used_info.kps = np.array(info['kps']) if info['kps'] else None
                                face_to_save = get_aligned_face(frame, used_info)
                                video_faces_found = True

                    face_crops_temp.append(face_to_save)
                    if is_real and used_info and face_to_save is not None:
                        current_video_metadata[i] = {
                            "bbox": used_info.bbox.tolist(),
                            "kps": used_info.kps.tolist() if hasattr(used_info, 'kps') else None
                        }

                # 파일 저장 (구글 드라이브로 직접 쓰기)
                if video_faces_found:
                    for idx, crop in enumerate(face_crops_temp):
                        if crop is not None:
                            cv2.imwrite(os.path.join(video_folder, f"f{idx:03d}_{label}.jpg"), crop)
                else:
                    for idx, frame in enumerate(frames_cache[:NUM_FRAMES]):
                        full_resized = resize_with_letterbox(frame, (256, 256))
                        cv2.imwrite(os.path.join(video_folder, f"f{idx:03d}_{label}.jpg"), full_resized)

                # 🔥 [중요] 매 영상 완료 시마다 JSON 실시간 업데이트
                if is_real:
                    metadata[video_id] = current_video_metadata
                    with open(JSON_DB_PATH, 'w') as f:
                        json.dump(metadata, f)

                cap.release()
            except Exception as e:
                print(f"❌ Error {v_name}: {e}")

In [8]:
# --- 샘플 확인 설정 ---
VERIFY_SAVE_PATH = "/content/drive/MyDrive/HECTO/Dataset/FF_Verification"

def run_visual_verification(detector):
    """
    각 조작 방식별로 첫 번째 Fake 영상을 선정하고,
    그에 대응하는 Real 영상을 자동으로 찾아 검증 리스트를 구성합니다.
    """
    print("🔍 샘플 검증 프로세스를 시작합니다...")
    if os.path.exists(VERIFY_SAVE_PATH):
        shutil.rmtree(VERIFY_SAVE_PATH)
    os.makedirs(VERIFY_SAVE_PATH, exist_ok=True)

    # 1. 데이터 리스트 가져오기
    real_videos, fake_videos_dict = get_id_matched_dataset()
    verify_list = []
    temp_metadata = {}

    # 2. 각 카테고리별로 Fake 샘플 하나씩 선정하고, 대응하는 Real도 세트로 추가
    # 이렇게 해야 Fake 처리 시 필요한 Real의 JSON 좌표가 반드시 존재하게 됩니다.
    for method in MANIPULATIONS:
        v_list = fake_videos_dict.get(method, [])
        if not v_list:
            print(f"⚠️ 경고: {method} 카테고리에 데이터가 없습니다.")
            continue

        # 첫 번째 가짜 영상 선택 (예: '751_752.mp4')
        fake_v_name = v_list[0]
        target_id = fake_v_name.split('_')[0] # '751' 추출

        # 해당 가짜 영상의 원본(Real) 파일명 생성 (예: '751.mp4')
        real_v_name = target_id + ".mp4"

        if real_v_name in real_videos:
            # Real 먼저 추가 (좌표 생성을 위해)
            verify_list.append(("Real", [real_v_name], f"original_sequences/youtube/{COMPRESSION}/videos/", 0, True))
            # 그 다음 Fake 추가 (좌표 재활용을 위해)
            verify_list.append((f"Fake/{method}", [fake_v_name], f"manipulated_sequences/{method}/{COMPRESSION}/videos/", 1, False))
        else:
            print(f"⚠️ 경고: {method}의 타겟 ID {target_id}에 해당하는 원본 영상을 찾을 수 없습니다.")

    # 3. 위에서 구성된 리스트 순차 처리
    for category, v_names, sub_path, label, is_real in verify_list:
        v_name = v_names[0]
        video_id = v_name.split('.')[0]
        target_id = video_id.split('_')[0] if not is_real else video_id

        # 영상별 개별 폴더 생성
        video_folder = os.path.join(VERIFY_SAVE_PATH, category, video_id)
        os.makedirs(video_folder, exist_ok=True)

        print(f"📸 {category} 처리 중: {video_id}")
        video_url = SERVER_URL + sub_path + v_name

        with tempfile.NamedTemporaryFile(suffix='.mp4') as temp_v:
            try:
                urllib.request.urlretrieve(video_url, temp_v.name)
                cap = cv2.VideoCapture(temp_v.name)
                for i in range(5): # 5프레임씩 검증
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i * 10)
                    ret, frame = cap.read()
                    if not ret: break

                    face_to_save = None
                    if is_real:
                        # [REAL] 직접 검출 및 좌표 저장
                        faces = detector.get(frame)
                        if faces:
                            # score가 None인 경우를 대비해 0.0 처리
                            used_info = sorted(faces, key=lambda x: (getattr(x, 'score', 0.0) or 0.0), reverse=True)[0]
                            face_to_save = get_aligned_face(frame, used_info)
                            if target_id not in temp_metadata: temp_metadata[target_id] = []
                            temp_metadata[target_id].append({
                                "bbox": used_info.bbox.tolist(),
                                "kps": used_info.kps.tolist() if hasattr(used_info, 'kps') else None
                            })
                    else:
                        # [FAKE] 저장된 좌표 재활용
                        if target_id in temp_metadata and i < len(temp_metadata[target_id]):
                            class FaceInfo: pass
                            info = temp_metadata[target_id][i]
                            f_obj = FaceInfo()
                            f_obj.bbox = np.array(info['bbox'])
                            f_obj.kps = np.array(info['kps']) if info['kps'] else None
                            face_to_save = get_aligned_face(frame, f_obj)

                    if face_to_save is not None:
                        cv2.imwrite(os.path.join(video_folder, f"f{i:03d}_{label}.jpg"), face_to_save)
                cap.release()
            except Exception as e:
                print(f"   -> ❌ {video_id} 에러: {e}")

    print(f"\n✅ 검증 완료! 구글 드라이브 폴더를 확인하세요: {VERIFY_SAVE_PATH}")

In [9]:
# --- [주의] FINAL_SAVE_PATH가 4번째 셀에서 정의되어 있어야 합니다 ---
# FINAL_SAVE_PATH = "/content/drive/MyDrive/HECTO/Dataset/FF_Frames_Final"

# 1. 환경 준비 (폴더 삭제 로직 제거 - 이어하기를 위해)
if not os.path.exists(FINAL_SAVE_PATH):
    os.makedirs(FINAL_SAVE_PATH, exist_ok=True)
    print(f"✅ 새 저장 디렉토리 생성: {FINAL_SAVE_PATH}")
else:
    print(f"ℹ️ 기존 디렉토리를 유지합니다. (이어하기 모드)")

# 2. 데이터 리스트 준비
real_videos, fake_videos_dict = get_id_matched_dataset()

# 3. 본 작업 시작
print("\n🎬 [1/2] Real 영상 처리 및 JSON 좌표 DB 구축 시작...")
real_sub_path = f"original_sequences/youtube/{COMPRESSION}/videos/"
process_and_extract_combined(real_videos, real_sub_path, "Real", 0, detector, is_real=True)

print("\n🎬 [2/2] Fake 영상 ID 매칭 처리 시작 (좌표 재활용)...")
for method, v_list in fake_videos_dict.items():
    fake_sub_path = f"manipulated_sequences/{method}/{COMPRESSION}/videos/"
    process_and_extract_combined(v_list, fake_sub_path, f"Fake/{method}", 1, detector, is_real=False)

print("\n🎉 모든 전처리가 완료되었습니다. 구글 드라이브를 확인하세요.")

✅ 새 저장 디렉토리 생성: /content/drive/MyDrive/HECTO/Dataset/FF_Frames_Final
서버에서 Fake 매칭 리스트를 가져오는 중...
✅ 최종 리스트 확정: Real 1000개 / Fake 총 1000개

🎬 [1/2] Real 영상 처리 및 JSON 좌표 DB 구축 시작...


Processing Real: 100%|██████████| 1000/1000 [2:13:13<00:00,  7.99s/it]



🎬 [2/2] Fake 영상 ID 매칭 처리 시작 (좌표 재활용)...


Processing Fake/NeuralTextures: 100%|██████████| 250/250 [32:08<00:00,  7.71s/it]


🎉 모든 전처리가 완료되었습니다. 구글 드라이브를 확인하세요.
